分词就是将句子、段落、文章这种长文本，分解为以字词为单位的数据结构，方便后续的处理分析工作。

In [ ]:
import re  # 基于正则表达式进行分词


class Tokenizer:
    def __init__(self, vocab, unk_token="<|unk|>", endof_token="<|endoftext|>"):
        if vocab.get(unk_token) is None:
            raise ValueError(f"Vocabulary must contain unk_token '{unk_token}'")
        if vocab.get(endof_token) is None:
            raise ValueError(f"Vocabulary must contain endof_token '{endof_token}'")

        self._unk_token = unk_token
        self._endof_token = endof_token
        self._str_to_int = vocab
        self._int_to_str = {v: k for k, v in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [items.strip() for items in preprocessed if items.strip()]
        return [
            self._str_to_int.get(item, self._str_to_int[self._unk_token])
            for item in preprocessed
        ]

    def decode(self, tokens):
        text = " ".join(self._int_to_str[token] for token in tokens)
        text = re.sub(r'\s+([,.:;?_!"()\'])', r"\1", text)
        return text

我们使用这个文件来构建我们的词表，并使用它来实例化一个分词器。

In [ ]:
with open("asserts/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total length of the text:", len(raw_text))
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [items.strip() for items in preprocessed if items.strip()]
print("Total length of the preprocessed text:", len(preprocessed))

preprocessed.extend(["<|unk|>", "<|endoftext|>"])
vocab = {token: integer for integer, token in enumerate(sorted(set(preprocessed)))}
print("Total length of the vocabulary:", len(vocab))

In [ ]:
tokenizer = Tokenizer(vocab)
text1 = "Hello, do you like tea"
text2 = """"It's the last he painted, you know, "
Mrs. Gisburn said with pardonable pride."""

ids = tokenizer.encode("<|endoftext|>".join([text1, text2]))
print(ids)
text = tokenizer.decode(ids)
print(text)

接下来我们使用 gpt2 的 tokenizer 来进行分词。

In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

text1 = "Hello, do you like tea"
text2 = """"It's the last he painted, you know, "
Mrs. Gisburn said with pardonable pride."""
idx = tokenizer.encode(
    "<|endoftext|>".join([text1, text2]), allowed_special={"<|endoftext|>"}
)
print(idx)
text = tokenizer.decode(idx)
print(text)

接下来我们来实现一个简单的数据集，用于训练模型。
对于 GPT 来说，它需要的输入是连续的字词，而输出是下一个字词。
所以我们需要以此构建数据集

In [ ]:
enc_text = tokenizer.encode(raw_text)
context_size = 10
for i in range(1, context_size + 1):
    context = enc_text[:i]
    desired = enc_text[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

我们用 DataSet 和 DataLoader 来实现这个数据集。

In [ ]:
from torch.utils.data import Dataset


class GPTDataset(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        # 在初始化的时候按照上面的思路做拆分
        self._chunks = []
        token_ids = tokenizer.encode(txt)
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = tuple(token_ids[i : i + max_length])
            output_chunk = tuple(token_ids[i + 1 : i + max_length + 1])
            self._chunks.append((input_chunk, output_chunk))

    def __len__(self):
        return len(self._chunks)

    def __getitem__(self, index):
        return self._chunks[index]

我们模拟一下数据集的使用

In [ ]:
dataset = GPTDataset(raw_text, tokenizer, max_length=10, stride=1)
for i in dataset[:3]:
    print(tokenizer.decode(i[0]), "---->", tokenizer.decode(i[1]))

我们做一个函数来创建一个 dataloader

In [ ]:
from torch.utils.data import DataLoader


def create_dataloader(
    text,
    batch_size=4,
    max_length=256,
    stride=128,
    shuffle=True,
    drop_last=True,
    num_workers=0,
):
    return DataLoader(
        GPTDataset(text, tokenizer, max_length=max_length, stride=stride),
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers,
    )

In [ ]:
dataloader = create_dataloader(raw_text, batch_size=1, max_length=4, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
input, output = first_batch
print(input, output)